#### Import required library


In [194]:
import numpy as np
import random
import math
from sklearn.model_selection import train_test_split

#### Loading the data as x and y

In [195]:
data = np.load('../data/problem_7.npz')
x_real= data['x']
y_real = data['y']
print(f"x_real shape: {x_real.shape}")
print(f"y_real shape: {y_real.shape}")

x_real shape: (2, 5000)
y_real shape: (5000,)


#### Spliting the data into two sets


In [196]:
x_train, x_test, y_train, y_test = train_test_split(x_real.T, y_real, test_size=0.2, random_state=42)

#### defining constants, functions and variables

In [197]:
OPERATORS = ['+', '-', '*', '/']  # Basic binary operators
FUNCTIONS = ['sin', 'cos', 'log', 'tan', 'exp', 'sqrt']        # Example unary operators
VARIABLES = [f"x{i}" for i in range(x_real.shape[0])] #used to have the variables according to the dataset
# FUNCTIONS = [np.sin, np.cos, np.log, np.tan, np.exp, np.sqrt]
def random_constant():
    return round(random.uniform(-5, 5), 2)

#### Generate Random formula

In [198]:
def generate_random_expr(max_depth, prob_function=0.5):
    
    if max_depth == 0:
        return f"x{random.randint(0, 1)}"
    # Base case or random stop condition
    if max_depth == 0 or random.random() < 0.3:
        # Choose either a variable or a constant
        if random.random() < 0.5:
            return random.choice(VARIABLES)
        else:
            return random_constant()
        # return random.choice(VARIABLES + [random_constant()])

    # With some probability, pick a unary operator like sin, cos
    if random.random() < prob_function:
        func = random.choice(FUNCTIONS)
        child = generate_random_expr(max_depth - 1, prob_function)
        return f"{func} ({child})"
    else:
        # Otherwise, pick a binary operator
        op = random.choice(OPERATORS)
        left_child = generate_random_expr(max_depth - 1, prob_function)
        right_child = generate_random_expr(max_depth - 1, prob_function)
        left_child = str(left_child)  # Convert to string if not already
        right_child = str(right_child)  # Convert to string if not already
        return f"{left_child} {op} {right_child}"

#### Transform and evaluation of the formula

In [199]:

# Safe versions of functions (handles edge cases like negative inputs to log and sqrt)
def safe_log(x):
    if x <= 0:
        return None  # Invalid input for log
    return np.log(x)

def safe_sqrt(x):
    if x < 0:
        return None  # Invalid input for sqrt
    return np.sqrt(x)

# Define other operations like sin, cos, tan, exp, etc.
def safe_sin(x):
    return np.sin(x)

def safe_cos(x):
    return np.cos(x)

def safe_tan(x):
    return np.tan(x)

def safe_exp(x):
    return np.exp(x)

# Apply these safe functions to the transformation process
def transform_formula(formula):
    # Replace the string-based functions with numpy functions
    if isinstance(formula, str):
        formula = formula.replace("log", "safe_log")  # Replace 'log' with safe_log
        formula = formula.replace("sqrt", "safe_sqrt")  # Replace 'sqrt' with safe_sqrt
        formula = formula.replace("sin", "np.sin")  # Replace 'sin' with np.sin
        formula = formula.replace("cos", "np.cos")  # Replace 'cos' with np.cos
        formula = formula.replace("tan", "safe_tan")  # Replace 'tan' with safe_tan
        formula = formula.replace("exp", "np.exp")  # Replace 'exp' with np.exp
    return formula

def evaluate_expr(expr, x_values):
    # Define the mapping of x-values to variables like x0, x1, etc.
    variables = ['x0', 'x1']
    
    # Remove spaces and tokenize the expression
    if isinstance(expr, str):
        expr = expr.replace(" ", "")
    
    # Try evaluating the expression with x_values
    try:
        # Use Python eval function to evaluate the formula with the safe functions
        return eval(expr, {"x0": x_values[0], "x1": x_values[1], 
                           "np": np, "safe_log": safe_log, "safe_sqrt": safe_sqrt, "safe_tan": safe_tan})
    except Exception as e:
        # print(f"Error in expression: {expr}, {e}")
        return np.nan  # Return NaN for invalid formulas



#### Fitness computation (MSE)

In [200]:


def compute_fitness(expr, x_values, y_values, complexity_weight=0.01):
    y_pred = []
    for x_values in x_train:  # Iterate over each column (each sample)
        prediction = evaluate_expr(expr, x_values)
        if prediction is not None:
            y_pred.append(prediction)
        else:
            y_pred.append(float('inf'))  # Assign infinity if evaluation failed
    y_pred = np.array(y_pred)
    mse = np.mean((y_pred - y_values) ** 2)
    return mse 


### fitness filter

In [201]:
def filter_valid_fitness(fitness_scores, population):
    valid_population = []
    valid_fitness_scores = []
    
    for i, fitness in enumerate(fitness_scores):
        if np.isfinite(fitness):  # Check if the fitness score is a valid number
            valid_population.append(population[i])
            valid_fitness_scores.append(fitness)
    
    return valid_population, valid_fitness_scores

### Selection


##### Tournment


In [202]:
# Tournament selection function: Selects the best individual from a random subset
def tournament_selection(population, fitness_scores, num_parents, tournament_size=3):
    # print("population in tournment selection is ", population)
    parents = []
    for _ in range(num_parents):
        # Randomly select a subset of the population
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_population = [population[i] for i in tournament_indices]
        tournament_scores = [fitness_scores[i] for i in tournament_indices]
        
        # Select the best individual (with the lowest fitness) from the tournament
        best_index = tournament_scores.index(min(tournament_scores))
        parents.append(tournament_population[best_index])
    
    return parents

In [203]:
def run_selection(population_size, fitness_scores, population):
    # Filter out invalid fitness scores
    valid_population, valid_fitness_scores = filter_valid_fitness(fitness_scores, population)
    
    if len(valid_population) == 0:
        print("No valid fitness scores. Exiting the selection process.")
        return None, None  # Return None if there are no valid individuals
    
    num_parents = population_size // 2  # Half of the population is selected for the next generation
    
    # Tournament Selection
    # print("Tournament Selection Parents:")
    tournament_parents = tournament_selection(valid_population, valid_fitness_scores, num_parents)
    # for i, parent in enumerate(tournament_parents, start=1):
    #     print(f"Parent {i}: {parent}")
    
    return tournament_parents

#### Crossover of 2 parents

In [204]:
# def crossover(parent1, parent2):
#     # Choose a random point in both parents to split
#     point1 = random.randint(0, len(parent1) - 1)
#     point2 = random.randint(0, len(parent2) - 1)
    
#     # Split both parents at the chosen points
#     part1 = parent1[:point1]
#     part2 = parent2[point2:]
    
#     # Combine the parts to form a new child
#     child = part1 + part2
#     return child

In [205]:
# def crossover_population(population, num_to_select):
#     offspring = []
    
#     # Select a random sample of parents from the population
#     selected_parents = random.sample(population, num_to_select)
    
#     # Perform crossover on selected parents in pairs
#     for i in range(0, len(selected_parents), 2):
#         parent1 = selected_parents[i]
#         parent2 = selected_parents[i + 1]
        
#         # Generate offspring using crossover
#         child = crossover(parent1, parent2)
#         offspring.append(child)
    
#     return offspring

In [206]:
import ast
import random


def parse_formula_to_tree(formula):
    """
    Parse a mathematical formula into an abstract syntax tree (AST).
    """
    return ast.parse(formula, mode='eval').body

def tree_to_formula(tree):
    """
    Convert an AST back to a formula string.
    """
    return ast.unparse(tree)


In [207]:
def select_random_subtree(tree):
    """
    Randomly select a subtree from the given AST node.
    """
    if isinstance(tree, ast.BinOp) or isinstance(tree, ast.UnaryOp):
        children = [tree.left, tree.right] if isinstance(tree, ast.BinOp) else [tree.operand]
        if random.random() < 0.5:
            return tree  # Return the current tree
        return select_random_subtree(random.choice(children))
    return tree  # Return leaf nodes (constants or variables) directly

def replace_subtree(tree, target, replacement):
    """
    Replace a target subtree in the tree with the replacement subtree.
    """
    if tree == target:
        return replacement
    if isinstance(tree, ast.BinOp):
        tree.left = replace_subtree(tree.left, target, replacement)
        tree.right = replace_subtree(tree.right, target, replacement)
    elif isinstance(tree, ast.UnaryOp):
        tree.operand = replace_subtree(tree.operand, target, replacement)
    return tree

def crossover(parent1, parent2):
    """
    Perform subtree crossover between two parent formulas.
    """
    tree1 = parse_formula_to_tree(parent1)
    # print("tree 1 output is ")
    # print(ast.dump(tree1, indent=4))
    tree2 = parse_formula_to_tree(parent2)
    
    # Select random subtrees
    subtree1 = select_random_subtree(tree1)
    subtree2 = select_random_subtree(tree2)
    
    # Swap subtrees
    offspring_tree1 = replace_subtree(tree1, subtree1, subtree2)
    offspring_tree2 = replace_subtree(tree2, subtree2, subtree1)
    
    # Convert back to formulas
    offspring1 = tree_to_formula(offspring_tree1)
    offspring2 = tree_to_formula(offspring_tree2)
    
    return offspring1, offspring2


In [208]:
def crossover_population(population, num_to_select):
    offspring = []
    selected_parents = random.sample(population, num_to_select)
    
    # Ensure even number of parents
    if len(selected_parents) % 2 != 0:
        selected_parents = selected_parents[:-1]
    
    # Perform crossover in pairs
    for i in range(0, len(selected_parents), 2):
        parent1 = selected_parents[i]
        parent2 = selected_parents[i + 1]
        child1, child2 = crossover(parent1, parent2)
        offspring.extend([child1, child2])
    
    return offspring


#### Mutation in the formula

In [209]:
import random
import ast

def mutate_formula(formula, mutation_rate=0.5):
    """Applies mutation to a formula with a given probability."""
    if random.random() > mutation_rate:
        return formula  # No mutation occurs

    # Convert the formula to an AST tree
    try:
        tree = ast.parse(formula, mode='eval')
    except Exception as e:
        print(f"Error parsing formula '{formula}' to AST: {e}")
        return formula  # Return the original formula if parsing fails

    # Mutate the tree
    mutated_tree = mutate_tree(tree, mutation_rate, VARIABLES)
    return ast.unparse(mutated_tree)  # Convert back to string formula

def mutate_tree(tree, mutation_rate, variables):
    """Mutates an AST tree."""
    mutations = 0
    for node in ast.walk(tree):
        # Mutate constants
        if isinstance(node, ast.Constant) and random.random() < mutation_rate:
            node.value = random.uniform(-10, 10)  # Replace with a random number
            mutations += 1

        # Mutate binary operators
        elif isinstance(node, ast.BinOp) and random.random() < mutation_rate:
            node.op = random.choice([ast.Add(), ast.Sub(), ast.Mult(), ast.Div()])
            mutations += 1

        # Mutate function calls (e.g., sin, cos, log)
        elif isinstance(node, ast.Call) and random.random() < mutation_rate:
            if isinstance(node.func, ast.Name):
                node.func.id = random.choice(["sin", "cos", "tan", "log", "exp", "sqrt"])
                mutations += 1

        # Mutate the variables themselves (e.g., change x0 to x1)
        elif isinstance(node, ast.Name) and random.random() < mutation_rate:
            if variables is not None:
                node.id = random.choice(variables)  # Use the passed variables list
                mutations += 1

    if mutations == 0:
        # If no mutations were made, force one mutation to ensure change
        node = random.choice(list(ast.walk(tree)))
        if isinstance(node, ast.Constant):
            node.value = random.uniform(-10, 10)
        elif isinstance(node, ast.BinOp):
            node.op = random.choice([ast.Add(), ast.Sub(), ast.Mult(), ast.Div()])
        elif isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            node.func.id = random.choice(["sin", "cos", "tan", "log", "exp", "sqrt"])
        elif isinstance(node, ast.Name):
            node.id = random.choice(VARIABLES)

    return tree



#### Genetic algorithm

In [211]:
# Parameters
num_generations = 50
population_size = 20
num_to_select = 10  # Half of the population is selected for the next generation
mutation_rate = 0.3

# Initial population
population = [generate_random_expr(max_depth=4) for _ in range(population_size)]

# Initialize variables to store the best formula and fitness score across all generations
best_formula = None
best_fitness = float('inf')  # Set to infinity for minimization, or -inf for maximization

# For logging and debugging purposes, we can track the best fitness score in each generation
best_fitness_scores_per_generation = []

for generation in range(num_generations):
    # print(f"\nGeneration {generation + 1}/{num_generations}")

    # 1. **Evaluate Fitness Scores for Current Population**
    transformed_formula = [transform_formula(formula) for formula in population]
    fitness_scores = [compute_fitness(formula, x_train, y_train) for formula in transformed_formula]

    # # Print the fitness scores (optional, for debugging)
    # for i, (formula, score) in enumerate(zip(population, fitness_scores)):
    #     print(f"Formula {i + 1}: {formula} | Fitness = {score}")

    # Track the best fitness score in this generation (optional)
    best_fitness_scores_per_generation.append(min(fitness_scores))  # Assuming lower fitness is better (in case of minimization)

    # 2. **Select Tournament Parents**
    tournament_parents = run_selection(population_size, fitness_scores, population)
    tournament_fitness_scores = [compute_fitness(transform_formula(formula), x_train, y_train) for formula in tournament_parents]

    # 3. **Crossover to Generate Offspring**
    offspring = crossover_population(tournament_parents, int(len(tournament_parents)/2))

    # # Print offspring after crossover (optional)
    # print("\nOffspring after crossover:")
    # for i, child in enumerate(offspring, start=1):
    #     print(f"Child {i}: {child}")

    # 4. **Mutation on the Offspring**
    mutated_offspring = [mutate_formula(child, mutation_rate) for child in offspring]

    # # Print offspring after crossover and mutation (optional)
    # print("\nOffspring after crossover and mutation:")
    # for i, child in enumerate(mutated_offspring, start=1):
    #     print(f"Child {i}: {child}")

    # 5. **Evaluate Fitness for Mutated Offspring**
    mutated_fitness_scores = [compute_fitness(transform_formula(formula), x_train, y_train) for formula in mutated_offspring]
    # print("\nFitness scores for mutated offspring:")
    # for i, (formula, score) in enumerate(zip(mutated_offspring, mutated_fitness_scores)):
    #     print(f"Mutated Child {i}: Fitness = {score}")

    # 6. **Combine Parents and Offspring for Selection**
    combined_population = tournament_parents + mutated_offspring
    combined_fitness_scores = tournament_fitness_scores + mutated_fitness_scores

    # print("\nCombined population:")
    # for i, (formula, score) in enumerate(zip(combined_population, combined_fitness_scores)):
    #     print(f"Formula {i + 1}: {formula} | Fitness = {score}")

    assert len(combined_population) == len(combined_fitness_scores), "Population and fitness scores do not match in length!"

    # 7. **Select the Best Individuals for the Next Generation**
    next_generation_parents = run_selection(population_size, combined_fitness_scores, combined_population)

    # print("\nSelected parents for the next generation:")
    # for i, parent in enumerate(next_generation_parents, start=1):
    #     print(f"Parent {i}: {parent}")

    # 8. **Generate the Next Generation Population**
    offspring_next_gen = crossover_population(next_generation_parents, population_size // 2)

    # print("\nOffspring after crossover (next generation):")
    # for i, child in enumerate(offspring_next_gen, start=1):
    #     print(f"Child {i}: {child}")

    # 9. **Mutation on the Next Generation**
    mutated_next_gen = [mutate_formula(child, mutation_rate) for child in offspring_next_gen]

    # print("\nOffspring after crossover and mutation (next generation):")
    # for i, child in enumerate(mutated_next_gen, start=1):
    #     print(f"Child {i}: {child}")

    # **Update Population for the Next Generation**
    population = mutated_next_gen  # Update population to the next generation

    # Combine fitness scores of parents and offspring
    combined_fitness_scores_all = fitness_scores + mutated_fitness_scores
    combined_population_all = population + mutated_offspring
    combined_population_all=transform_formula(combined_population_all)
    # Find the index of the best fitness score
    current_best_fitness = min(combined_fitness_scores_all)

    # If the current best fitness is better than the previous best, update
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        # Find the index of the formula with the best fitness score
        best_index = combined_fitness_scores_all.index(best_fitness)
        best_formula = combined_population_all[best_index]

# After all generations, track the best fitness score seen across all generations
print("\nBest formula across all generations:")
print(f"Best Fitness = {best_fitness}")
print(f"Best Formula = {best_formula}")

# If you'd like to see the best fitness score across generations
print("\nBest fitness scores across generations:")
for i, score in enumerate(best_fitness_scores_per_generation):
    print(f"Generation {i + 1}: Best Fitness = {score}")



Best formula across all generations:
Best Fitness = 609.8482524916744
Best Formula = 2.7 - x0 + exp(x1) + x1 * x1 / sqrt(1.75) + (2.7 - x0 + exp(x1)) / exp(x1) + (2.7 - x0 + exp(x1)) / exp(x1)

Best fitness scores across generations:
Generation 1: Best Fitness = 685.7266403406926
Generation 2: Best Fitness = nan
Generation 3: Best Fitness = nan
Generation 4: Best Fitness = 619.6960133981762
Generation 5: Best Fitness = 609.8482524916744
Generation 6: Best Fitness = 612.1416392478212
Generation 7: Best Fitness = 646.3306235703839
Generation 8: Best Fitness = nan
Generation 9: Best Fitness = 621.9945968723479
Generation 10: Best Fitness = 616.8958489873166
Generation 11: Best Fitness = 616.8958489873166
Generation 12: Best Fitness = 616.8958489873166
Generation 13: Best Fitness = 650.280951962416
Generation 14: Best Fitness = 622.1936063446462
Generation 15: Best Fitness = 622.1936063446462
Generation 16: Best Fitness = 622.1936063446462
Generation 17: Best Fitness = nan
Generation 18: 